# Getting the reflectivity values from the matched bins

In [3]:
import wradlib as wrl
import numpy as np
import matplotlib.pyplot as pl
import numpy as np
import pickle
import os

import pandas as pd
import datetime as dt

In [4]:
# Getting back the matched points from the previous notebook:
with open('./output/GRGRmatchedpoints.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    [rs_matched1_el0, thetas_matched1_el0, rs_matched2_el0, thetas_matched2_el0] = pickle.load(f)

In [5]:
SUB_fname = './DATA/TAG-20120806-171547-02-Z.nc'
TAG_fname = './DATA/TAG-20120806-171547-02-Z.nc'

PIA_fname = './DATA/PIA_dp_TAG-20120806-171547-02.hdf5'

SUB_BBF_fname = './DATA/SUB_qual_02-ZH_120km_r500m_BBF.hdf5'
TAG_BBF_fname = './DATA/TAG_qual_02-ZH_120km_r500m_BBF.hdf5'

In [6]:
pia_dp, att = wrl.io.from_hdf5(PIA_fname)

SUB_BBF_qual, att = wrl.io.from_hdf5(SUB_BBF_fname)
TAG_BBF_qual, att = wrl.io.from_hdf5(TAG_BBF_fname)

### Calculate Q_PIA

Based on equation from Friedrich et al. (2006)

\begin{equation}
Q_{att} = \left\{\begin{matrix}
1 & \text{for} & K_{r,s} < K_{min}\\ 
0 & \text{for} & K_{r,s} > K_{max}\\ 
\frac{K_{max} - K_{r,s}}{K_{max} - K_{min}} & \text{else}, & 
\end{matrix}\right\}
\end{equation}

In [7]:
Kmax = 10.0
Kmin = 1.0

TAG_PIA_qual = np.empty(pia_dp.shape)*np.nan
TAG_PIA_qual[pia_dp > Kmax] = 0
TAG_PIA_qual[pia_dp < Kmin] = 1
TAG_PIA_qual[(pia_dp>=Kmin)&(pia_dp<=Kmax)] = (Kmax-pia_dp[(pia_dp>=Kmin) & (pia_dp<=Kmax)])/(Kmax-Kmin)

In [8]:
Q_SUB = SUB_BBF_qual
Q_TAG = TAG_BBF_qual * TAG_PIA_qual

In [9]:
# fig = pl.figure(figsize=(11,4))

# ax1 = fig.add_subplot(1,2,1, aspect='equal')
# ax1,pm1 = wrl.vis.plot_ppi(SUB_BBF, ax=ax1, cmap=pl.cm.Blues_r)
# cb = pl.colorbar(pm1)
# pl.title('$Q_{SUB,BBF}$')

# ax2 = fig.add_subplot(1,2,2, aspect='equal')
# ax2,pm2 = wrl.vis.plot_ppi(TAG_BBF, ax=ax2, cmap=pl.cm.Blues_r)
# cb = pl.colorbar(pm2)
# pl.title('$Q_{TAG,BBF}$')

In [10]:
elev = '02'

if elev=='02':
    rs_matched1 = rs_matched1_el0
    thetas_matched1 = thetas_matched1_el0
    rs_matched2 = rs_matched2_el0
    thetas_matched2 = thetas_matched2_el0

try:
    dataSUB_el0, metaSUB_el0 = wrl.io.read_edge_netcdf(SUB_fname)
    dataTAG_el0, metaTAG_el0 = wrl.io.read_edge_netcdf(TAG_fname)
    
    pia_dp, att = wrl.io.from_hdf5(PIA_fname)
except:
    print('o',end='')

dataSUB_overlap_el0 = dataSUB_el0[thetas_matched1, rs_matched1]
dataTAG_overlap_el0 = dataTAG_el0[thetas_matched2, rs_matched2]

Q_SUB_overlap = Q_SUB[thetas_matched1, rs_matched1]
Q_TAG_overlap = Q_TAG[thetas_matched2, rs_matched2]

emptydata = np.empty((len(dataSUB_overlap_el0.ravel()), 10))
emptydata[:] = np.nan

df = pd.DataFrame(emptydata, columns=['sub_date','tag_date','time_diff','elev',
                                      'theta','r','sub_overlap','tag_overlap',
                                      'Q_TAG','Q_SUB'])

for j in df.index:
    df.set_value(j, 'sub_date', metaSUB_el0['time'])
    df.set_value(j, 'tag_date', metaTAG_el0['time'])
    df.set_value(j, 'time_diff', abs(metaTAG_el0['time']-metaSUB_el0['time']).seconds)
    df.set_value(j, 'elev',elev)
    df.set_value(j, 'theta', thetas_matched1_el0[j])
    df.set_value(j, 'r', rs_matched1_el0[j])
    df.set_value(j, 'sub_overlap', dataSUB_overlap_el0[j])
    df.set_value(j, 'tag_overlap', dataTAG_overlap_el0[j])
    df.set_value(j, 'Q_TAG', Q_TAG_overlap[j])
    df.set_value(j, 'Q_SUB', Q_SUB_overlap[j])

# SAVE TO FILE
# set path to save processed data
savepath = './output/'

dbfilename = 'SUBTAG_' + \
             'sub_'+ dt.datetime.strftime(metaSUB_el0['time'], '%Y%m%d_%H%M%S') + \
             '_tag_'+ dt.datetime.strftime(metaTAG_el0['time'], '%Y%m%d_%H%M%S') + \
             '.hdf5'    

try:
    df_nonan = df.loc[(~np.isnan(df.sub_overlap)) & (~np.isnan(df.tag_overlap))]
    df_nonan.to_hdf(os.path.join(savepath, dbfilename), 'dataframe')
except:
    raise
if len(df_nonan)==0:
    print('-',end='')
else:
    print('+',end='')
print('')

print('Done!')

C:\Anaconda\envs\wradlib35_2\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Anaconda\envs\wradlib35_2\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Anaconda\envs\wradlib35_2\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Anaconda\envs\wradlib35_2\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Anaconda\envs\wradlib35_2\lib\site-packages\ipykernel_launcher.py:35: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Anacond

+
Done!


C:\Anaconda\envs\wradlib35_2\lib\site-packages\pandas\core\generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->datetime,key->block1_values] [items->['sub_date', 'tag_date']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
